Exploratory data analysis(EDA )

Цель этапа - cформулировать предположения и гипотезы для построения модели. Проверить качество данных и очистить их. Определиться с параметрами модели.

In [832]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re
from itertools import combinations
from scipy.stats import ttest_ind

pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок

dataset = pd.read_csv('data.csv')

In [710]:
# Получаем информацию о датафрейме
display(dataset.head(10))
dataset.info()

,status,private pool,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,PrivatePool,MlsId,target
0,Active,NaN,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,NaN,611019,"$418,000"
1,for sale,NaN,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,NaN,201916904,"$310,000"
2,for sale,NaN,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,yes,FR19221027,"$2,895,000"
3,for sale,NaN,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,NaN,14191809,"$2,395,000"
4,for sale,NaN,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,NaN,861745,"$5,000"
5,for sale,NaN,townhouse,1624 S Newkirk St,NaN,"{'atAGlanceFacts': [{'factValue': '1920', 'fac...",NaN,Philadelphia,"[{'rating': [], 'data': {'Distance': [], 'Grad...",897 sqft,19145,2 Beds,PA,2.0,NaN,NaN,PAPH847006,"$209,000"
6,Active,NaN,Florida,552 Casanova Ct,NaN,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",NaN,POINCIANA,"[{'rating': ['3', '3', '1', 'NR'], 'data': {'D...","1,507",34759,NaN,FL,One,NaN,NaN,S5026943,"181,500"
7,Active,NaN,NaN,6094 Mingle Dr,NaN,"{'atAGlanceFacts': [{'factValue': '1976', 'fac...",NaN,Memphis,"[{'rating': ['4', '2', '2'], 'data': {'Distanc...",NaN,38115,NaN,TN,NaN,NaN,NaN,10063506,"68,000"
8,Active,NaN,Single Family Home,11182 Owl Ave,2,"{'atAGlanceFacts': [{'factValue': '1970', 'fac...",NaN,Mason City,"[{'rating': ['2', '2', '4', '7', '4', 'NR'], '...",3588,50401,3,IA,NaN,NaN,NaN,190988,"$244,900"
9,NaN,NaN,Single Family,8612 Cedar Plains Ln,3,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Houston,"[{'rating': ['4/10', '3/10', '2/10'], 'data': ...","1,930",77080,3,TX,2.0,NaN,NaN,73968331,"$311,995"


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 377185 entries, 0 to 377184
Data columns (total 18 columns):
status          337267 non-null object
private pool    4181 non-null object
propertyType    342452 non-null object
street          377183 non-null object
baths           270847 non-null object
homeFacts       377185 non-null object
fireplace       103115 non-null object
city            377151 non-null object
schools         377185 non-null object
sqft            336608 non-null object
zipcode         377185 non-null object
beds            285903 non-null object
state           377185 non-null object
stories         226470 non-null object
mls-id          24942 non-null object
PrivatePool     40311 non-null object
MlsId           310305 non-null object
target          374704 non-null object
dtypes: object(18)
memory usage: 51.8+ MB


In [711]:
# Постчитаем количество пропущенных занчений для каждого из столбцов 
dataset.isna().sum()

status           39918
private pool    373004
propertyType     34733
street               2
baths           106338
homeFacts            0
fireplace       274070
city                34
schools              0
sqft             40577
zipcode              0
beds             91282
state                0
stories         150715
mls-id          352243
PrivatePool     336874
MlsId            66880
target            2481
dtype: int64

In [712]:
dataset['status'].value_counts()

for sale                  156104
Active                    105207
For sale                   43465
foreclosure                 6426
New construction            5475
                           ...  
Coming soon: Nov 11.           1
Pending Backups Wanted         1
Coming soon: Dec 25.           1
Coming soon: Nov 26.           1
Coming soon: Nov 12.           1
Name: status, Length: 159, dtype: int64

In [713]:
dataset['status'] = dataset['status'].str.replace(r'.*for sale*', 'For sale', regex= True)

In [714]:
dataset['status'].value_counts()

For sale                199569
Active                  105207
foreclosure               6426
New construction          5475
Pending                   4702
                         ...  
Coming soon: Nov 17.         1
Coming soon: Nov 25.         1
Coming soon: Oct 24.         1
Coming soon: Dec 27.         1
Coming soon: Nov 12.         1
Name: status, Length: 158, dtype: int64

In [715]:
dataset.target.values.astype(str)

array(['$418,000', '$310,000', '$2,895,000', ..., '$528,000', '$34,500',
       '$204,900'], dtype='<U18')

In [716]:
# меняем формат солбца на строку
dataset = dataset.astype({'target': np.str})

In [717]:
# замена всех символов, кроме цифр
dataset.target = dataset.target.apply(lambda x: x.replace('$', ''))
dataset.target = dataset.target.apply(lambda x: x.replace(',', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('+', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('/', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('-', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('m', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('n', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('o', ''))
dataset.target = dataset.target.apply(lambda x: x.replace(' ', ''))
dataset.target = dataset.target.apply(lambda x: x.replace('a', ''))

In [718]:
dataset.target

0          418000
1          310000
2         2895000
3         2395000
4            5000
           ...   
377180    1249000
377181     674999
377182     528000
377183      34500
377184     204900
Name: target, Length: 377185, dtype: object

In [719]:
dataset.target.values.astype(str)

array(['418000', '310000', '2895000', ..., '528000', '34500', '204900'],
      dtype='<U9')

In [720]:
# проверка на остатки символов
full_set=set()
for s in [set(str(p)) for p in dataset.target.values]:
    full_set=full_set.union(s)

In [721]:
full_set

{'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [722]:
pd.to_numeric(dataset.target)

0          418000.0
1          310000.0
2         2895000.0
3         2395000.0
4            5000.0
            ...    
377180    1249000.0
377181     674999.0
377182     528000.0
377183      34500.0
377184     204900.0
Name: target, Length: 377185, dtype: float64

In [723]:
dataset[dataset.status=='For sale'].target.mean()

inf

In [724]:
# смотрим на часто встречающиеся значением
dataset['target'].value_counts()

           2481
225000     1806
275000     1650
250000     1644
350000     1641
           ... 
161770        1
1776250       1
257765        1
224052        1
155992        1
Name: target, Length: 34233, dtype: int64

In [725]:
# заполняем пропуски константой,самым часто часто встречающимся занчением
dataset['target'].fillna(225000, inplace=True)

In [726]:
# Замена NaN на значение 1-ого индекса, т.е. на 225 000
dataset['target'].fillna(dataset['target'].value_counts().index[1],inplace=True)

In [727]:
# проверяем, что пропуски заполнены
dataset['target'].isna().sum()

0

In [728]:
dataset[dataset.status=='For sale'].target.max()

'9999999'

In [729]:
dataset[dataset.status=='Active'].target.max()

'999999'

In [730]:
dataset[dataset.status=='For sale'].target.min()

''

In [731]:
dataset[dataset.status=='Activ'].target.max()

nan

In [732]:
# заполняем пропуски константой,самым часто часто встречающимся занчением
dataset['status'].fillna('For sale', inplace=True)

In [733]:
# проверяем, что пропуски заполнены
dataset['status'].isna().sum()

0

In [734]:
dataset['private pool'].value_counts()

Yes    4181
Name: private pool, dtype: int64

In [735]:
dataset['private pool'].fillna('No', inplace=True)

In [736]:
# думаю, что нужно будет заменить на чиловые значения 1 и 2
dataset['private pool'].value_counts()

No     373004
Yes      4181
Name: private pool, dtype: int64

In [737]:
dataset['PrivatePool'].value_counts()

yes    28793
Yes    11518
Name: PrivatePool, dtype: int64

In [738]:
dataset['PrivatePool'] = dataset['PrivatePool'].str.replace(r'.*yes*', 'Yes', regex= True)

In [739]:
dataset['PrivatePool'].value_counts()

Yes    40311
Name: PrivatePool, dtype: int64

In [740]:
dataset['PrivatePool'].fillna('No', inplace=True)

In [741]:
dataset['PrivatePool'].value_counts()

No     336874
Yes     40311
Name: PrivatePool, dtype: int64

In [742]:
dataset['Pool']=dataset['private pool'].astype(str) +'_'+ dataset['PrivatePool'].astype(str)

In [743]:
dataset['Pool'].value_counts()

No_No     332693
No_Yes     40311
Yes_No      4181
Name: Pool, dtype: int64

In [744]:
dataset.Pool = dataset.Pool.apply(lambda x: x.replace('No_No', 'No'))
dataset.Pool = dataset.Pool.apply(lambda x: x.replace('No_Yes', 'Yes'))
dataset.Pool = dataset.Pool.apply(lambda x: x.replace('Yes_No', 'Yes'))

In [745]:
dataset['Pool'].value_counts()

No     332693
Yes     44492
Name: Pool, dtype: int64

In [746]:
# удаляем столбцы, на сонове которых создан новый признак
dataset.drop(['private pool','PrivatePool'],axis=1,inplace=True)

In [747]:
dataset

,status,propertyType,street,baths,homeFacts,fireplace,city,schools,sqft,zipcode,beds,state,stories,mls-id,MlsId,target,Pool
0,Active,Single Family Home,240 Heather Ln,3.5,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",Gas Logs,Southern Pines,"[{'rating': ['4', '4', '7', 'NR', '4', '7', 'N...",2900,28387,4,NC,NaN,NaN,611019,418000,No
1,For sale,single-family home,12911 E Heroy Ave,3 Baths,"{'atAGlanceFacts': [{'factValue': '2019', 'fac...",NaN,Spokane Valley,"[{'rating': ['4/10', 'None/10', '4/10'], 'data...","1,947 sqft",99216,3 Beds,WA,2.0,NaN,201916904,310000,No
2,For sale,single-family home,2005 Westridge Rd,2 Baths,"{'atAGlanceFacts': [{'factValue': '1961', 'fac...",yes,Los Angeles,"[{'rating': ['8/10', '4/10', '8/10'], 'data': ...","3,000 sqft",90049,3 Beds,CA,1.0,NaN,FR19221027,2895000,Yes
3,For sale,single-family home,4311 Livingston Ave,8 Baths,"{'atAGlanceFacts': [{'factValue': '2006', 'fac...",yes,Dallas,"[{'rating': ['9/10', '9/10', '10/10', '9/10'],...","6,457 sqft",75205,5 Beds,TX,3.0,NaN,14191809,2395000,No
4,For sale,lot/land,1524 Kiscoe St,NaN,"{'atAGlanceFacts': [{'factValue': '', 'factLab...",NaN,Palm Bay,"[{'rating': ['4/10', '5/10', '5/10'], 'data': ...",NaN,32908,NaN,FL,NaN,NaN,861745,5000,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377180,For sale,Single Family,20800 NE 23rd Ave,6.0,"{'atAGlanceFacts': [{'factValue': '1990', 'fac...",NaN,Miami,"[{'rating': ['10/10', '5/10'], 'data': {'Dista...","4,017",33180,5,FL,0.0,NaN,A10702700,1249000,Yes
377181,For sale,condo,3530 N Lake Shore Dr #4B,3 Baths,"{'atAGlanceFacts': [{'factValue': '1924', 'fac...",NaN,Chicago,"[{'rating': ['1/10', '5/10', '7/10'], 'data': ...","2,000 sqft",60657,3 Beds,IL,9.0,NaN,10374233,674999,No
377182,For sale,single-family home,15509 Linden Blvd,3 Baths,"{'atAGlanceFacts': [{'factValue': '1950', 'fac...",NaN,Jamaica,"[{'rating': ['5/10', '4/10'], 'data': {'Distan...","1,152 sqft",11434,3 Beds,NY,2,NaN,NaN,528000,No
377183,For sale,NaN,7810 Pereida St,NaN,"{'atAGlanceFacts': [{'factValue': None, 'factL...",NaN,Houston,"[{'rating': ['NA', 'NA', 'NA'], 'data': {'Dist...",NaN,77028,"8,479 sqft",TX,NaN,NaN,NaN,34500,No


In [748]:
dataset['propertyType'].value_counts()

single-family home                                                                92206
Single Family                                                                     62869
Single Family Home                                                                31728
condo                                                                             25968
lot/land                                                                          20552
                                                                                  ...  
Custom, Ranch, Traditional                                                            1
1 Story, Cape Cod                                                                     1
Lake House, Mid-Century Modern                                                        1
Historical/Conservation District, Single Detached, Early American, Traditional        1
Bungalow, Craftsman, Tudor                                                            1
Name: propertyType, Length: 1280

In [749]:
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*single-family home*', 'Single Family', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Single Family Home*', 'Single Family', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*condo*', 'Condo', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*lot/land*', 'Land', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*townhouse*', 'Townhouse', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*multi-family*', 'Multi Family', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Multi-Family Home*', 'Multi Family', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Detached, One Story*', 'Single Detached, Traditional', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Colonial*', 'Condo', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*mobile/manufactured*', 'Mobile / Manufactured', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Condo*', 'Apartment', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*coop*', 'Cooperative', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*apartment*', 'Apartment', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Two Story*', '2 Story', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*One Story*', '1 Story', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*2 Stories*', '2 Story', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*2 Story, Traditional*', '2 Story', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Contemporary/Modern*', 'Contemporary', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Apartment/Townhome/Row Home/Co-Op*', 'Apartment', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Single Detached, Traditional*', 'Single Detached', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Contemporary, Traditional*', 'Contemporary', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Mfd/Mobile Home*', 'Mobile / Manufactured', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Other Style*', 'Other', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*1 Story, Traditional*', '1 Story', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Apartment/Townhome, Traditional*', 'Apartment', regex= True)
dataset['propertyType'] = dataset['propertyType'].str.replace(r'.*Apartment/Townhome*', 'Apartment', regex= True)

In [750]:
# можно будет дорпботать ещё, позднее
dataset['propertyType'].value_counts().head(20)

Single Family            186803
Apartment                 52821
Land                      31486
Townhouse                 18402
Multi Family              12195
Traditional                5913
Single Detached            3974
Cooperative                3801
Mobile / Manufactured      3476
Contemporary               3316
2 Story                    2468
1 Story                    2269
High Rise                  1823
Ranch                      1781
Transitional                560
Florida                     530
Other                       507
Bungalow                    442
Farms/Ranches               371
Garden Home                 305
Name: propertyType, dtype: int64

In [751]:
dataset['propertyType'].fillna('Single Family', inplace=True)

In [752]:
# проверяем, что пропуски заполнены
dataset['propertyType'].isna().sum()

0

In [762]:
# меняем формат солбца на строку
dataset = dataset.astype({'street': np.str})

In [764]:
dataset['street'] = dataset['street'].str.replace(r'.*Address Not Disclosed*', 'Unknown Address', regex= True)
dataset['street'] = dataset['street'].str.replace(r'.*Undisclosed Address*', 'Unknown Address', regex= True)
dataset['street'] = dataset['street'].str.replace(r'.*Address Not Available*', 'Unknown Address', regex= True)
dataset['street'] = dataset['street'].str.replace(r'.*undisclosed Address*', 'Unknown Address', regex= True)

In [766]:
dataset.street = dataset.street.apply(lambda x: x.replace(')', ''))

In [831]:
# смотрим значения
dataset['street'].value_counts().head(20)

Unknown Address                1827
2103 E State Hwy 21              57
11305 Gulf Fwy                   54
17030 Youngblood Rd.             38
NE 58th Cir                      34
9470 Lancaster Rd. SW            32
1 Palmer Dr                      27
8426 Terrace Valley Circle       25
9845 Basil Western Rd NW         25
6320 SW 89th Court Road          24
8447 SW 99th Street Rd           22
5221 S. Zapata Hwy               20
Whitetail Trail                  19
Stone Bluff Drive                18
3423 Heather Garden Trail        17
3435 Heather Garden Trail        17
24423 Ferdossa Drive             17
2005 West Happy Valley Road      17
1365 Neihart Way                 17
Boncher Blvd                     17
Name: street, dtype: int64

In [768]:
# всего 2 пропуска в этом признаке
dataset['street'].fillna('Unknown Address', inplace=True)

In [769]:
# проверяем, что пропуски заполнены
dataset['street'].isna().sum()

0

In [774]:
# смотрим на значения
dataset['baths'].value_counts().head(30)

nan             106338
2 Baths          52466
3 Baths          35506
2                20452
2.0              16576
4 Baths          14764
3.0              10869
3                10113
Bathrooms: 2      9538
2.5               8113
Bathrooms: 3      6613
1                 6583
1.0               5930
5 Baths           5370
4.0               4593
2.5 Baths         4486
0                 3811
4                 3567
3.5               3455
Bathrooms: 1      3234
2 ba              2686
6 Baths           2604
Bathrooms: 4      2384
1,500             2209
3 ba              1893
1.5               1857
3.5 Baths         1821
2,000             1795
1,750             1723
3,000             1686
Name: baths, dtype: int64

In [786]:
# меняем формат солбца на строку
dataset = dataset.astype({'baths': np.str})

In [808]:
dataset.baths = dataset.baths.apply(lambda x: x.replace(',', '.'))
dataset.baths = dataset.baths.apply(lambda x: x.replace('~', '.'))
dataset.baths = dataset.baths.apply(lambda x: x.replace('+', '.'))

In [793]:
# проверка на остатки символов
b_set=set()
for s in [set(str(p)) for p in dataset.baths.values]:
    b_set=full_set.union(s)

In [794]:
b_set

{'.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

In [826]:
dataset['baths'] = dataset['baths'].str.replace(r'.*2 Baths*', '2', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*3 Baths*', '3', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*4 Baths*', '4', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*5 Baths*', '5', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*6 Baths*', '6', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*7 Baths*', '7', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*8 Baths*', '8', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*9 Baths*', '9', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*10 Baths*', '10', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*11 Baths*', '11', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*12 Baths*', '12', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*13 Baths*', '13', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*14 Baths*', '14', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*15 Baths*', '15', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*16 Baths*', '16', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*17 Baths*', '17', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*18 Baths*', '18', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*19 Baths*', '19', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*20 Baths*', '20', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*21 Baths*', '21', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*2.5 Baths*', '2.5', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*Bathrooms: 1*', '1', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*Bathrooms: 2*', '2', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*Bathrooms: 3*', '3', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*Bathrooms: 4*', '4', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*3 ba*', '3', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*1 ba*', '1', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*2 ba*', '2', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*4 ba*', '4', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*5 ba*', '5', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*6 ba*', '6', regex= True)
dataset['baths'] = dataset['baths'].str.replace(r'.*-- baths*', '2', regex= True)

In [829]:
# смотрим на значения!!!!!!!!!!!!!!!!
dataset['baths'].value_counts().tail(50)

2ths                     5
15.0                     4
14.0                     4
9.5                      4
10 ba                    4
24                       4
21                       3
4.1 Baths                3
23.0                     3
124                      3
10.5                     3
30 Baths                 3
40                       2
12.5                     2
27                       2
9.5.                     2
30                       2
12.5.                    2
24.0                     2
40 Baths                 2
22.0                     2
3ths                     2
26                       2
11.5.                    2
18.0                     2
20.0                     2
22                       2
121                      1
35                       1
29                       1
41.0                     1
116 / 116 / 116          1
1 / 1 / 1 / 1            1
34.0                     1
127                      1
3-1 / 2-2                1
14.5.                    1
1

In [ ]:
dataset['baths'].value_counts()

In [833]:
dataset['homeFacts'].value_counts()

{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': '', 'factLabel': 'Parking'}, {'factValue': '—', 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                                 7174
{'atAGlanceFacts': [{'factValue': None, 'factLabel': 'Year built'}, {'factValue': None, 'factLabel': 'Remodeled year'}, {'factValue': None, 'factLabel': 'Heating'}, {'factValue': None, 'factLabel': 'Cooling'}, {'factValue': None, 'factLabel': 'Parking'}, {'factValue': None, 'factLabel': 'lotsize'}, {'factValue': None, 'factLabel': 'Price/sqft'}]}                                      3532
{'atAGlanceFacts': [{'factValue': '', 'factLabel': 'Year built'}, {'factValue': '', 'factLabel': 'Remodeled year'}, {'factValue': '', 'factLabel': 'Heating'}, {'factValue': '', 'factLabel': 'Cooling'}, {'factValue': ''

In [837]:
dataset['fireplace'].value_counts().tail(10)

Central Vacuum, Humidifier, Redecorated, Utility Connection, Walk-In Closets, Wet Bar          1
Ceiling Fan, Hot Tub                                                                           1
FAMILYRM, Master Bedroom, See Thru                                                             1
Air Filter, Ceiling Fan, Central Vacuum, In-Law Quarters, Walk-In Closets, Wet Bar             1
Brick, Gas Log, Fireplace Insert, Two                                                          1
Central Vacuum, Extra Closets, Storage, Utility Connection, Walk-In Closets                    1
3+ Fireplace, Fireplace Family Rm, Fireplace Living Rm, Fireplace Master Bdr, Gas Fireplace    1
Living Room, See Thru                                                                          1
Air Filter, Ceiling Fan, Extra Closets, In-Law Quarters, Storage, Utility Connection           1
Extra Closets, In-Law Quarters, Water Filter, Wet Bar, Wood Burning FP                         1
Name: fireplace, dtype: int64

In [840]:
dataset['city'].value_counts().tail(50)

Wachee                       1
Forest Park                  1
Britt                        1
Largo                        1
Freedom                      1
Outside Area (Outside Ca)    1
Quito Ecuador                1
China                        1
Chinatown                    1
LAKE BUENA VISTA             1
Jackson Township             1
New Palestine                1
Midkiff                      1
Point Charlotte              1
Palm Beach Gardens           1
Hyden                        1
Savage                       1
Beulah                       1
Belllingham                  1
Superior Twp                 1
Mabank                       1
Mem                          1
Gettysburg                   1
St. Johns                    1
St george                    1
Pittsfield Township          1
Taylor Landing               1
Indian Orchard               1
Baldwin Hills                1
Conewango Valley             1
SAND LAKE                    1
Mayfield Hts                 1
Lower Ea

In [ ]:
dataset['schools'].value_counts()

In [841]:
dataset['sqft'].value_counts()

0                                          11854
1,200 sqft                                   839
1,000 sqft                                   654
1,100 sqft                                   573
1,800 sqft                                   563
                                           ...  
6,564                                          1
5,247                                          1
23,870 sqft                                    1
Total interior livable area: 8,203 sqft        1
7,155                                          1
Name: sqft, Length: 25405, dtype: int64

In [ ]:
dataset['zipcode'].value_counts()

In [842]:
dataset['beds'].value_counts()

3 Beds         53459
4 Beds         35418
3              31406
2 Beds         26362
4              20030
               ...  
8,102 sqft         1
5,566 sqft         1
31.37 acres        1
7.66 acres         1
6.41 acres         1
Name: beds, Length: 1184, dtype: int64

In [843]:
dataset['state'].value_counts()

FL    115449
TX     83786
NY     24479
CA     23386
NC     21862
TN     18340
WA     13826
OH     12588
IL      8939
NV      8482
GA      6705
CO      6404
PA      5561
MI      5161
DC      4674
AZ      3347
IN      3328
OR      2789
MA      1516
UT      1325
MD      1090
VT       868
MO       866
VA       801
WI       452
NJ       436
ME       259
IA       242
KY        90
OK        49
MS        40
SC        28
MT         7
DE         5
OS         1
AL         1
BA         1
OT         1
Fl         1
Name: state, dtype: int64

In [844]:
dataset['stories'].value_counts()

1.0                         67454
2.0                         55283
1                           23086
2                           18146
3.0                         11275
                            ...  
58.0                            1
3 Story, Multi Level            1
Site Built, Townhouse           1
3.000                           1
1-2 Stories, 3-4 Stories        1
Name: stories, Length: 348, dtype: int64

In [ ]:
dataset['mls-id'].value_counts()

In [ ]:
dataset['MlsId'].value_counts()